In [50]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

# Итерационные методы решения линейных систем. 
## Варинат 9

In [51]:
let a = matrix [
    [12.78572; 1.534675; -3.947418]
    [1.534675; 9.709232; 0.918435]
    [-3.947418; 0.918435; 7.703946]
]

let b = vector [9.60565; 7.30777; 4.21575]

In [52]:
let accurateSolution = a.Solve b

printfn "Точное решение СЛАУ: %A" accurateSolution

Точное решение СЛАУ: seq [1.00000036; 0.5000044523; 1.000000238]


**Задание 2.** Преобразовать исходную систему к системе вида x = H * x + g, где H = E − D^(−1) * A, g = D^(−1) * b. Здесь D - диагональная матрица, у которой на диагонали находятся диагональные элементы матрицы A. Вычислить ||H||∞.

In [53]:
let transform (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let n = matrix.RowCount 

    // H = E - D^(-1) * A   
    let h = DenseMatrix.init n n (fun i j -> if i = j then 0. else - matrix.[i, j] / matrix.[i, i])
       
    // g = D ^ (-1) * b     
    let g = DenseVector.init n (fun i -> freeVector.[i] / matrix.[i, i])

    h, g

In [54]:
let (h, g) = transform a b

printfn "Матрица H = %O" h
printfn "Вектор g = %O" g
printfn "||H||_inf = %f" <| h.InfinityNorm()

Матрица H = DenseMatrix 3x3-Double
        0   -0,12003   0,308736
-0,158063          0  -0,094594
 0,512389  -0,119216          0

Вектор g = DenseVector 3-Double
 0,75128
0,752662
 0,54722

||H||_inf = 0.631605


**Задание 3.** Найти априорную оценку того k, при котором ||x∗ − xk||∞ < ε, ε = 0.001

In [55]:
let epsilon = 0.001

let calculatePrioriEstimation (h: Matrix<float>) (g: Vector<float>) (firstEstiomation: Vector<float>) iterationsCount =
    h.InfinityNorm() ** (float iterationsCount) * firstEstiomation.InfinityNorm() +  
    h.InfinityNorm() ** (float iterationsCount) / (1. - h.InfinityNorm()) * g.InfinityNorm()

let getIterationsCount accuracy (h: Matrix<float>) (g: Vector<float>) (firstEstiomation: Vector<float>)  = 
    let mutable iterationsCount = 1
    while calculatePrioriEstimation h g firstEstiomation iterationsCount >= accuracy do 
        iterationsCount <- iterationsCount + 1

    int iterationsCount

In [56]:
let firstEstimation = DenseVector.zero<float> h.RowCount

let approximateIterationCount = getIterationsCount epsilon h g firstEstimation
let prioriEstimation = calculatePrioriEstimation h g firstEstimation approximateIterationCount
printfn "Для получения решения с априорной оценкой %f (< %f) необходимо %i итераций"
    <| prioriEstimation
    <| epsilon
    <| approximateIterationCount

Для получения решения с априорной оценкой 0.000828 (< 0.001000) необходимо 17 итераций


**Задание 4.** Вычислить решение методом простой итерации с точностью ε = 0.001. Сравнить требуемое фактическое число итераций с априорным значением k. Вывести фактическую погрешность, апостериорную оценку, априорную оценку. Уточнить последнее приближение по Люстернику. Вывести его фактическую погрешность.

In [57]:
let getMaxEigenValue (matrix: Matrix<float>) = 
    matrix.Evd().EigenValues.AbsoluteMaximum().Magnitude

// Достаточное условие сходимости: ||H|| < 1
// Необходимое и достаточное условие сходимости: ρ(H) < 1
let solveIterative accuracy (h: Matrix<float>) (g: Vector<float>) step = 
    let mutable previousEstimation = firstEstimation
    let mutable currentEstimation: Vector<float> = step h g previousEstimation
    let mutable iterationCount = 1

    let calculatePosteriorEstimation (previousEstimation: Vector<float>) (currentEstimation: Vector<float>) = 
        h.InfinityNorm() / (1. - h.InfinityNorm()) * (currentEstimation - previousEstimation).InfinityNorm()

    while calculatePosteriorEstimation previousEstimation currentEstimation >= accuracy do
        previousEstimation <- currentEstimation
        currentEstimation <- step h g currentEstimation
        iterationCount <- iterationCount + 1

    let optimizeLusternik (previousEstimation: Vector<float>) (currentEstimation: Vector<float>) = 
        let maxEigen = getMaxEigenValue h
        if maxEigen > 1. then
            currentEstimation
        else 
            previousEstimation + (1. / (1. - maxEigen)) * (currentEstimation - previousEstimation)
    
    currentEstimation,  
    {|
        FactIterationCount = iterationCount
        PosteriorEstimation = calculatePosteriorEstimation previousEstimation currentEstimation
        LusternikOptimization = optimizeLusternik previousEstimation currentEstimation
    |} 

In [58]:
let solveSimpleIteration accuracy (h: Matrix<float>) (g: Vector<float>) =
    solveIterative accuracy h g <| fun (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) ->
        h * previousX + g

In [59]:
let (solutionSimpleIteration, infoSimpleIteration) = solveSimpleIteration epsilon h g
printfn "Решение системы методом простой итерации: %O" solutionSimpleIteration
printfn "Априорное число итераций: %O" approximateIterationCount
printfn "Фактическое число итераций: %O" infoSimpleIteration.FactIterationCount
printfn "Фактическая погрешность решения: %O" <| (solutionSimpleIteration - accurateSolution).InfinityNorm()
printfn "Априорная оценка решения: %O" prioriEstimation
printfn "Апостериорная оценка решения: %O" infoSimpleIteration.PosteriorEstimation
printfn "Решение с уточнением по Люстернику: %O" infoSimpleIteration.LusternikOptimization
printfn "Фактическая погрешность решения по Люстернику: %O" <| (infoSimpleIteration.LusternikOptimization - accurateSolution).InfinityNorm()

Решение системы методом простой итерации: DenseVector 3-Double
0,999681
0,500186
0,999633

Априорное число итераций: 17
Фактическое число итераций: 10
Фактическая погрешность решения: 0,0003668999994144162
Априорная оценка решения: 0,0008276837452870448
Апостериорная оценка решения: 0,000880313980431832
Решение с уточнением по Люстернику: DenseVector 3-Double
0,999941
0,499999
 1,00007

Фактическая погрешность решения по Люстернику: 7,435848701287107E-05


**Задание 5.** Вычислить решение систем методом Зейделя с точностью ε = 0.001.

In [60]:
let solveSeidel accuracy (h: Matrix<float>) (g: Vector<float>) =
    solveIterative accuracy h g <| fun (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) ->
        let x = DenseVector.zero<float> previousX.Count
        for i = 0 to x.Count - 1 do
            x.[i] <- h.[i, *] * x.[0 .. i - 1].ToColumnMatrix().Stack(previousX.[i .. x.Count - 1].ToColumnMatrix()).Column(0) + g.[i]
        x

In [61]:
let (solutionSeidel, infoSeidel) = solveSeidel epsilon h g
printfn "Решение системы методом простой итерации: %O" solutionSeidel
printfn "Априорное число итераций: %O" approximateIterationCount
printfn "Фактическое число итераций: %O" infoSeidel.FactIterationCount
printfn "Фактическая погрешность решения: %O" <| (solutionSeidel - accurateSolution).InfinityNorm()
printfn "Априорная оценка решения: %O" prioriEstimation
printfn "Апостериорная оценка решения: %O" infoSeidel.PosteriorEstimation
printfn "Решение с уточнением по Люстернику: %O" infoSeidel.LusternikOptimization
printfn "Фактическая погрешность решения по Люстернику: %O" <| (infoSeidel.LusternikOptimization - accurateSolution).InfinityNorm()

Решение системы методом простой итерации: DenseVector 3-Double
0,999859
0,500061
0,999921

Априорное число итераций: 17
Фактическое число итераций: 6
Фактическая погрешность решения: 0,00014160843960953073
Априорная оценка решения: 0,0008276837452870448
Апостериорная оценка решения: 0,0008576139159981784
Решение с уточнением по Люстернику: DenseVector 3-Double
1,00029
0,49989
1,00016

Фактическая погрешность решения по Люстернику: 0,0002882716110590433


**Задание 7.** Получить решение системы Ax = b методом верхней релаксации с точностью ε = 0.001. В качестве критерия использовать фактическую погрешность.

In [62]:
// Метод будет сходиться, если матрица самосопряженная, положительно-определенная и кроме того 0 < q < 2.
let solveUpperRelaxation accuracy (h: Matrix<float>) (g: Vector<float>) =
    let q = 2. / (1. + sqrt (1. - (getMaxEigenValue h) ** 2.)) 
    
    solveIterative accuracy h g <| fun (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) ->
        let x = DenseVector.zero<float> previousX.Count
        for i = 0 to x.Count - 1 do
            x.[i] <- 
                previousX.[i] + 
                q * (h.[i, 0 .. i - 1] * x.[0 .. i - 1] + 
                    h.[i, i + 1 .. x.Count - 1] * previousX.[i + 1 .. x.Count - 1] - 
                    previousX.[i] + g.[i])
        x

In [63]:
let (solutionUpperRelaxation, infoUpperRelaxation) = solveUpperRelaxation epsilon h g
printfn "Решение системы методом простой итерации: %O" solutionUpperRelaxation
printfn "Априорное число итераций: %O" approximateIterationCount
printfn "Фактическое число итераций: %O" infoUpperRelaxation.FactIterationCount
printfn "Фактическая погрешность решения: %O" <| (solutionUpperRelaxation - accurateSolution).InfinityNorm()
printfn "Априорная оценка решения: %O" prioriEstimation
printfn "Апостериорная оценка решения: %O" infoUpperRelaxation.PosteriorEstimation
printfn "Решение с уточнением по Люстернику: %O" infoUpperRelaxation.LusternikOptimization
printfn "Фактическая погрешность решения по Люстернику: %O" <| (infoUpperRelaxation.LusternikOptimization - accurateSolution).InfinityNorm()

Решение системы методом простой итерации: DenseVector 3-Double
0,999969
0,500021
0,999989

Априорное число итераций: 17
Фактическое число итераций: 5
Фактическая погрешность решения: 3,1450158422541286E-05
Априорная оценка решения: 0,0008276837452870448
Апостериорная оценка решения: 0,0005429939199840995
Решение с уточнением по Люстернику: DenseVector 3-Double
1,00024
0,49999
 1,0001

Фактическая погрешность решения по Люстернику: 0,00024072622477411265
